## <span style="color: blue;">1️⃣ <b><u>Import Libraries</u></b> </span>

In [285]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime as dt
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG
from pandas_datareader import data as web
import talib

## <span style="color: green;">2️⃣ <b><u>Pull Data From Yahoo Finance</u></b></span>

In [279]:
# Define the function to get stock data
def get_stock_data(ticker, start_date, end_date, interval):
    df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    df.dropna(inplace=True)
    
    # Drop the 'Adj Close' column if it exists
    if 'Adj Close' in df.columns:
        df.drop(columns=['Adj Close'], inplace=True)
    
    # Ensure single-level column names
    df.columns = df.columns.get_level_values(0)  # Flatten columns
    df.columns.name = None  # Remove the name attribute from columns

    # Reset index to include Date column
    df.reset_index(inplace=True)
    df.set_index('Date', inplace=True)
    
    # Ensure columns are in correct order
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    return df

## 3️⃣ <span style="color: red;"><b><u>Trading Strategies</u></b></span>

### <b> 1. </b> Moving Average Ribbon Strategy [Parv]

In [54]:
def moving_average_ribbon_strategy(df):
    short_window = 10
    long_window = 50
    df['Short_MA'] = df['Close'].rolling(window=short_window, min_periods=1).mean()
    df['Long_MA'] = df['Close'].rolling(window=long_window, min_periods=1).mean()
    
    df['Signal'] = 0
    df['Signal'][short_window:] = np.where(df['Short_MA'][short_window:] > df['Long_MA'][short_window:], 1, 0)
    
    return df

### <b> 2. </b> SuperTrend Strategy [Parv]

In [52]:
def supertrend_strategy(df, atr_period=7, multiplier=3):
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    tr = pd.DataFrame([high_low, high_close, low_close]).max(axis=0)
    atr = tr.rolling(atr_period).mean()
    
    df['Upper_Band'] = (df['High'] + df['Low']) / 2 + (multiplier * atr)
    df['Lower_Band'] = (df['High'] + df['Low']) / 2 - (multiplier * atr)
    df['Supertrend'] = np.nan
    
    for i in range(1, len(df)):
        if df['Close'][i] > df['Upper_Band'][i - 1]:
            df['Supertrend'][i] = df['Lower_Band'][i]
        elif df['Close'][i] < df['Lower_Band'][i - 1]:
            df['Supertrend'][i] = df['Upper_Band'][i]
        else:
            df['Supertrend'][i] = df['Supertrend'][i - 1]
    
    df['Signal'] = np.where(df['Close'] > df['Supertrend'], 1, 0)
    
    return df

### <b> 3. </b> Volatility Breakout Strategy [Parv]

In [ ]:
def volatility_breakout_strategy(df, lookback=20, multiplier=2):
    df['High_Max'] = df['High'].rolling(lookback).max()
    df['Low_Min'] = df['Low'].rolling(lookback).min()
    df['Range'] = df['High_Max'] - df['Low_Min']
    
    df['Buy_Level'] = df['Open'] + (multiplier * df['Range'].shift())
    df['Sell_Level'] = df['Open'] - (multiplier * df['Range'].shift())
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['Close'] > df['Buy_Level'], 1, df['Signal'])
    df['Signal'] = np.where(df['Close'] < df['Sell_Level'], -1, df['Signal'])
    
    return df

### <b> 4. </b> Money Flow Index (MFI) Strategy [Anand]

In [61]:
def mfi_strategy(df, period=14):
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    money_flow = typical_price * df['Volume']
    
    positive_flow = money_flow.copy()
    negative_flow = money_flow.copy()
    
    positive_flow[typical_price < typical_price.shift(1)] = 0
    negative_flow[typical_price > typical_price.shift(1)] = 0
    
    positive_mf = positive_flow.rolling(period).sum()
    negative_mf = negative_flow.rolling(period).sum()
    
    mfi = 100 - (100 / (1 + (positive_mf / negative_mf)))
    df['MFI'] = mfi
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['MFI'] < 20, 1, df['Signal'])
    df['Signal'] = np.where(df['MFI'] > 80, -1, df['Signal'])
    
    return df

### <b> 5. </b> Volume Price Trend (VPT) Strategy [Anand]

In [64]:
def vpt_strategy(df):
    vpt = ((df['Close'].pct_change() + 1).cumprod() - 1) * df['Volume']
    df['VPT'] = vpt.cumsum()
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['VPT'] > df['VPT'].shift(), 1, 0)
    
    return df

### <b> 6. </b> Heikin-Ashi Candlesticks Strategy [Anand]

In [67]:
def heikin_ashi_strategy(df):
    ha_df = df.copy()
    ha_df['Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['Open'] = (df['Open'].shift() + df['Close'].shift()) / 2
    ha_df['High'] = df[['Open', 'Close', 'High']].max(axis=1)
    ha_df['Low'] = df[['Open', 'Close', 'Low']].min(axis=1)
    
    df['HA_Close'] = ha_df['Close']
    df['HA_Open'] = ha_df['Open']
    
    df['Signal'] = 0
    df['Signal'] = np.where(df['HA_Close'] > df['HA_Open'], 1, df['Signal'])
    df['Signal'] = np.where(df['HA_Close'] < df['HA_Open'], -1, df['Signal'])
    
    return df

### <b> 7. </b> Renko Chart Strategy [Neel]

In [330]:
def calculate_renko(df, brick_size):
    df['Direction'] = None
    df['Renko'] = None
    for i in range(1, len(df)):
        if df['Close'][i] >= df['Close'][i-1] + brick_size:
            df.loc[df.index[i], 'Direction'] = 'Up'
            df.loc[df.index[i], 'Renko'] = df['Close'][i-1] + brick_size
        elif df['Close'][i] <= df['Close'][i-1] - brick_size:
            df.loc[df.index[i], 'Direction'] = 'Down'
            df.loc[df.index[i], 'Renko'] = df['Close'][i-1] - brick_size
    df.dropna(inplace=True)
    return df[['Renko']]

class RenkoStrategy(Strategy):
    brick_size = 2  # Define the size of the Renko bricks

    def init(self):
        # Use the pre-calculated renko data
        close_prices = self.data.Close
        self.renko = calculate_renko(pd.DataFrame({'Close': close_prices}), brick_size=self.brick_size)['Renko'].values

    def next(self):
        # Ensure Renko bricks are calculated
        if len(self.renko) < 2 or np.isnan(self.renko[-1]) or np.isnan(self.renko[-2]):
            return

        # Prevent selling if no position is held
        if not self.position and self.renko[-1] < self.renko[-2]:
            print("Ignoring sell signal because no position is held")
            return

        # Simple trading logic based on Renko bricks
        # Buy when a new Renko brick is formed upwards
        # Sell when a new Renko brick is formed downwards
        if self.renko[-1] > self.renko[-2]:
            print("Buying")
            self.buy()
        elif self.renko[-1] < self.renko[-2]:
            print("Selling")
            self.sell()

# Example usage with sample data
ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2020-01-01'

# Download historical data for backtesting
df = get_stock_data(ticker, start_date, end_date, '1d')

# Backtest the strategy
bt = Backtest(df, RenkoStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot()

[*********************100%***********************]  1 of 1 completed
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal because no position is held
Ignoring sell signal

C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p8569', ...)

### <b> 8. </b> Mean Reversion with Bollinger Bands and RSI Strategy [Neel]

In [328]:
class MeanReversionBBandRSI(Strategy):
    bb_window = 20
    rsi_window = 14
    upper_bb = 2
    lower_bb = 2

    def init(self):
        # Calculate Bollinger Bands
        close_prices = self.data.Close
        self.upper_band, self.middle_band, self.lower_band = self.I(talib.BBANDS, close_prices, timeperiod=self.bb_window, nbdevup=self.upper_bb, nbdevdn=self.lower_bb)
        
        # Calculate RSI
        self.rsi = self.I(talib.RSI, close_prices, timeperiod=self.rsi_window)

    def next(self):
        # Ensure that both Bollinger Bands and RSI are valid before proceeding
        if np.isnan(self.upper_band[-1]) or np.isnan(self.lower_band[-1]) or np.isnan(self.rsi[-1]):
            return

        # Prevent selling if no position is held
        if not self.position and self.data.Close[-1] > self.upper_band[-1] and self.rsi[-1] > 70:
            # print("Ignoring sell signal because no position is held")
            return

        # Trading logic based on Bollinger Bands and RSI
        if self.data.Close[-1] < self.lower_band[-1] and self.rsi[-1] < 30:
            # print("Buying")
            self.buy()
        elif self.data.Close[-1] > self.upper_band[-1] and self.rsi[-1] > 70:
            # print("Selling")
            self.sell()

# Example usage with sample data
ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2020-01-01'

# Download historical data for backtesting
df = get_stock_data(ticker, start_date, end_date, '1d')

# Backtest the strategy
bt = Backtest(df, MeanReversionBBandRSI, cash=10000, commission=.002)
stats = bt.run()
print(stats)
print(len(stats['_trades']))
bt.plot()

[*********************100%***********************]  1 of 1 completed
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2010-01-04 00:00:00
End                       2019-12-31 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   71.581876
Equity Final [$]                  34722.00545
Equity Peak [$]                  34912.407511
Return [%]                         247.220054
Buy & Hold Return [%]              860.492488
Return (Ann.) [%]                   13.278223
Volatility (Ann.) [%]               24.559391
Sharpe Ratio                         0.540658
Sortino Ratio                         0.88424
Calmar Ratio                         0.342879
Max. Drawdown [%]                  -38.725669
Avg. Drawdown [%]                   -4.022495
Max. Drawdown Duration      721 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       362.6131
Worst Trade [%]                     247.11047
Avg. Trade [%]                    

C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p8384', ...)

### <b> 9. </b> Dual Moving Average Strategy [Neel]

In [320]:
class DualMovingAverage(Strategy):
    short_window = 50
    long_window = 200

    def init(self):
        close_prices = self.data.Close
        self.short_ma = self.I(talib.SMA, close_prices, timeperiod=self.short_window)
        self.long_ma = self.I(talib.SMA, close_prices, timeperiod=self.long_window)

    def next(self):
        if np.isnan(self.short_ma[-1]) or np.isnan(self.long_ma[-1]):
            return

        # Prevent selling if there are no positions to sell
        if not self.position and crossover(self.long_ma, self.short_ma):
            print("Ignoring sell signal because no position is held")
            return

        if crossover(self.short_ma, self.long_ma):
            print("Buying")
            self.buy()
        elif crossover(self.long_ma, self.short_ma):
            print("Selling")
            self.sell()
            
# Example usage with sample data
ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2024-01-01'

# Download historical data for backtesting
df = get_stock_data(ticker, start_date, end_date, '1d')

# Backtest the strategy
bt = Backtest(df, DualMovingAverage, cash=10000, commission=.002)
stats = bt.run()
print(stats)
print(len(stats['_trades']))
bt.plot()

[*********************100%***********************]  1 of 1 completed
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Ignoring sell signal because no position is held
Buying
Selling
Buying
Selling
Buying
Selling
Buying
Selling
Buying
Start                     2010-01-04 00:00:00
End                       2023-12-29 00:00:00
Duration                   5107 days 00:00:00
Exposure Time [%]                   73.594549
Equity Final [$]                115377.074821
Equity Peak [$]                 117882.028816
Return [%]                        1053.770748
Buy & Hold Return [%]             2418.966407
Return (Ann.) [%]                   19.122808
Volatility (Ann.) [%]               29.256994
Sharpe Ratio                         0.653615
Sortino Ratio                        1.161018
Calmar Ratio                         0.493845
Max. Drawdown [%]                  -38.722315
Avg. Drawdown [%]                   -4.024485
Max. Drawdown Duration      721 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade

C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\neelp\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p6981', ...)

## <span style="color: purple;">4️⃣ <b><u>Backtesting</u></b></span>

In [21]:
# Function to backtest a given trading strategy
def backtest_strategy(strategy, df):
    bt = Backtest(df, strategy, cash=10000, commission=.002)
    stats = bt.run()
    bt.plot()
    return stats

## <span style="color: brown;">5️⃣ <u><b>Comparing with TD Mutual Fund</b></u></span>

In [40]:
# Function to compare trading strategy with a mutual fund
def compare_with_mutual_fund(strategy_results, mutual_fund_data):
    combined_data = pd.concat([strategy_results['_equity_curve']['Equity'], mutual_fund_data], axis=1)
    combined_data.columns = ['Strategy Equity', 'Mutual Fund']
    combined_data.plot(figsize=(12, 6), title='Strategy vs Mutual Fund Comparison')
    plt.ylabel('Equity')
    plt.show()

# Function to load mutual fund data
def get_mutual_fund_data(ticker, start_date, end_date, interval='1d'):
    return get_stock_data(ticker, start_date, end_date, interval)['Close']

# Example usage with a placeholder trading strategy
def trading_strategy(df):
    # Placeholder strategy - Buy and hold
    df['Signal'] = 1
    return df